Inpendent Variables:
    GDP per capita
    Human Development Index (HDI)
    Total Population
    Transparency Index

Dependent Variables:
    Total Cases per million
    Total Deaths per million
    People fully vaccinated per hundred
    
Other Data Columns:
    ISO Code
    Continent
    Location
    Date

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import gmaps
import requests
import gmaps.datasets
import ipywidgets as widgets
import seaborn as sns; sns.set(style='white')
import gmaps.geojson_geometries

In [2]:
#NEED TO FIND WAY TO GET THIS INTO REPO 
############# NEED TO ADD THESE TO GIT IGNORE ###########################
from configMarc import g_key
gmaps.configure(api_key=g_key)

In [3]:
# Study data files
owidCovidData = "Resources/owid-covid-data.csv"
transparencyData = "Resources/Transparency Index.csv"

CovidData = pd.read_csv(owidCovidData)
Transparency = pd.read_csv(transparencyData)
# Transparency

In [4]:
# eliminate extra columns
CovidClean = CovidData.copy()
CovidClean.drop([ 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'new_cases_per_million', 
                 'new_cases_smoothed_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 
                 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 
                 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests', 'total_tests_per_thousand', 
                 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 
                 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations', 'new_vaccinations_smoothed', 
                 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'new_vaccinations_smoothed_per_million', 'stringency_index',
                 'median_age', 'aged_65_older', 'aged_70_older', 'extreme_poverty', 'cardiovasc_death_rate',
                 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy'],
                 axis = 1, inplace = True)


# limit to max date
CovidClean = CovidClean[CovidClean['date'].isin(['2021-04-27'])]


# rename "Location" to "Country"
CovidClean.rename(columns={"location":"Country"}, inplace=True)

# rename everything else for ease
CovidClean.rename(columns={"total_cases_per_million":"Cases", "total_deaths_per_million": "Deaths", "people_fully_vaccinated_per_hundred":"Vaxxed", 
                           "population": "Population", "population_density": "Density","gdp_per_capita": "GDP", "human_development_index": "HDI"}, inplace=True)

# merge with Transparency (inner join)
CovidAll= pd.merge(CovidClean, Transparency, on=["Country"], how="inner")

CovidClean
CovidAll
CovidAll.to_csv("Resources/CovidWorkingData.csv", index = False)


In [72]:
CovidClean

,iso_code,continent,Country,date,Cases,Deaths,Vaxxed,Population,Density,GDP,HDI
428,AFG,Asia,Afghanistan,2021-04-27,1521.385,66.738,NaN,3.892834e+07,54.422,1803.987,0.511
868,OWID_AFR,NaN,Africa,2021-04-27,3373.910,90.104,0.36,1.340598e+09,NaN,NaN,NaN
1296,ALB,Europe,Albania,2021-04-27,45429.147,828.063,NaN,2.877800e+06,104.871,11803.431,0.795
1724,DZA,Africa,Algeria,2021-04-27,2767.186,73.544,NaN,4.385104e+07,17.348,13913.839,0.748
2146,AND,Europe,Andorra,2021-04-27,169818.158,1604.866,NaN,7.726500e+04,163.755,NaN,0.868
...,...,...,...,...,...,...,...,...,...,...,...
82874,VNM,Asia,Vietnam,2021-04-27,29.351,0.360,NaN,9.733858e+07,308.127,6171.884,0.704
83336,OWID_WRL,NaN,World,2021-04-27,19078.988,402.186,3.19,7.794799e+09,58.045,15469.207,0.737
83719,YEM,Asia,Yemen,2021-04-27,208.543,40.468,NaN,2.982597e+07,53.508,1479.147,0.470
84125,ZMB,Africa,Zambia,2021-04-27,4972.706,67.940,NaN,1.838396e+07,22.995,3689.251,0.584


In [6]:
CovidContinent = CovidClean.copy()
# CovidClean = CovidClean[CovidClean['iso_code'].isin(["OWID_AFR"])]
# CovidClean = CovidClean.isna(CovidClean["Density"])
CovidContinent.isna()['continent']

Continent = CovidContinent.loc[CovidContinent.isna()['continent']]
Continent

,iso_code,continent,Country,date,Cases,Deaths,Vaxxed,Population,Density,GDP,HDI
868,OWID_AFR,NaN,Africa,2021-04-27,3373.910,90.104,0.36,1.340598e+09,NaN,NaN,NaN
4436,OWID_ASI,NaN,Asia,2021-04-27,8186.914,108.743,1.22,4.639847e+09,NaN,NaN,NaN
26089,OWID_EUR,NaN,Europe,2021-04-27,59322.749,1345.065,7.89,7.486801e+08,NaN,NaN,NaN
26550,OWID_EUN,NaN,European Union,2021-04-27,68344.672,1532.895,8.13,4.449191e+08,NaN,NaN,NaN
36081,OWID_INT,NaN,International,2021-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56091,OWID_NAM,NaN,North America,2021-04-27,62992.638,1424.228,17.78,5.920722e+08,NaN,NaN,NaN
57505,OWID_OCE,NaN,Oceania,2021-04-27,1015.399,24.462,0.14,4.267781e+07,NaN,NaN,NaN
70659,OWID_SAM,NaN,South America,2021-04-27,56768.938,1528.191,5.21,4.307598e+08,NaN,NaN,NaN
83336,OWID_WRL,NaN,World,2021-04-27,19078.988,402.186,3.19,7.794799e+09,58.045,15469.207,0.737


In [69]:
CovidAll

,iso_code,continent,Country,date,Cases,Deaths,Vaxxed,Population,Density,GDP,HDI,Score,Rank
0,AFG,Asia,Afghanistan,2021-04-27,1521.385,66.738,NaN,38928341.0,54.422,1803.987,0.511,19.0,165.0
1,ALB,Europe,Albania,2021-04-27,45429.147,828.063,NaN,2877800.0,104.871,11803.431,0.795,36.0,104.0
2,DZA,Africa,Algeria,2021-04-27,2767.186,73.544,NaN,43851043.0,17.348,13913.839,0.748,36.0,104.0
3,AGO,Africa,Angola,2021-04-27,789.320,17.860,NaN,32866268.0,23.890,5819.495,0.581,27.0,142.0
4,ARG,South America,Argentina,2021-04-27,64279.722,1385.063,1.95,45195777.0,16.177,18933.907,0.845,42.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,VEN,South America,Venezuela,2021-04-27,6812.540,73.217,NaN,28435943.0,36.253,16745.022,0.711,15.0,176.0
171,VNM,Asia,Vietnam,2021-04-27,29.351,0.360,NaN,97338583.0,308.127,6171.884,0.704,36.0,104.0
172,YEM,Asia,Yemen,2021-04-27,208.543,40.468,NaN,29825968.0,53.508,1479.147,0.470,15.0,176.0
173,ZMB,Africa,Zambia,2021-04-27,4972.706,67.940,NaN,18383956.0,22.995,3689.251,0.584,33.0,117.0


In [ ]:
# See if there is substantial difference looking at only the first half of the pandemic

CovidAug20 = CovidData.copy()
CovidAug20.drop([ 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'new_cases_per_million', 
                 'new_cases_smoothed_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 
                 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 
                 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests', 'total_tests_per_thousand', 
                 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 
                 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations', 'new_vaccinations_smoothed', 
                 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'new_vaccinations_smoothed_per_million', 'stringency_index',
                 'median_age', 'aged_65_older', 'aged_70_older', 'extreme_poverty', 'cardiovasc_death_rate',
                 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy'],
                 axis = 1, inplace = True)


# limit to max date
CovidAug20 = CovidAug20[CovidAug20['date'].isin(['2020-08-31'])]

# rename "Location" to "Country"
CovidAug20.rename(columns={"location":"Country"}, inplace=True)

# rename everything else for ease
CovidAug20.rename(columns={"total_cases_per_million":"Cases", "total_deaths_per_million": "Deaths", "people_fully_vaccinated_per_hundred":"Vaxxed", 
                           "population": "Population", "population_density": "Density","gdp_per_capita": "GDP", "human_development_index": "HDI"}, inplace=True)

# merge with Transparency (inner join)
CovidAll820= pd.merge(CovidAug20, Transparency, on=["Country"], how="inner")

# rename index to location

CovidAug20
CovidAll820
CovidAll820.to_csv("Resources/CovidWorkingData820.csv", index = False)

In [ ]:
CovidAll820

In [ ]:
# Pair Plots as of 4/27/2021

# Use Seaborn to create a PairGrid
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'coor = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

g = sns.pairplot(CovidAll, corner=True)  # Set corner=True to plot only the lower triangle:
g.map_lower(corrfunc)
plt.show()

In [ ]:
# Pair Plots as of 8/31/2020

# Use Seaborn to create a PairGrid
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'coor = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

g = sns.pairplot(CovidAll820, corner=True)  # Set corner=True to plot only the lower triangle:
g.map_lower(corrfunc)
plt.show()

In [ ]:
# Pair Plots by continent
# Use Seaborn to create a PairGrid
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'coor = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

g = sns.pairplot(Continent, corner=True)  # Set corner=True to plot only the lower triangle:
g.map_lower(corrfunc)
plt.show()

### Playing with Maps

In [88]:
MapsFrame = CovidClean.copy()
#MapsFrame.dtypes
#MapsFrame["country"].unique()
#MapsFrame.head()

In [89]:
latlong_df = pd.read_csv("Resources/Country Coordinates.csv")
#latlong_df.head()

In [90]:
#TRY TO NOT RERUN THIS AFTER THE FIRST KERNAL RE-START BECAUSE IT MAKES YOU RESTART THE KERNAL EVERY TIME!
del latlong_df["Unnamed: 3"]
del latlong_df["Unnamed: 4"]
del latlong_df["Unnamed: 5"]
del latlong_df["Unnamed: 6"]
#latlong_df.head()

In [91]:
#In MapsFrame it is "Country" and in latlong_df it is "country"
latlong_df=latlong_df.rename(columns = {"country" : "Country"})
#latlong_df.head()

In [92]:
map_df=pd.merge(MapsFrame, latlong_df, on="Country", how = "inner")
#map_df.tail()
#197 rows, 196 countries,  13 columns

In [13]:
#map_df["Country"].unique()

In [77]:
# locations = map_df[["latitude","longitude"]]
# weights = map_df["Cases"]
# fig1 = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=400, point_radius = 1)
# fig1.add_layer(heat_layer)
# #plt.savefig("../output_data/Map1")
# #fig1

In [ ]:
# locations = map_df[["latitude","longitude"]]
# weights = map_df["Deaths"]
# fig2 = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=400, point_radius = 1)
# fig2.add_layer(heat_layer)
# #plt.savefig("../output_data/Map1")
# #fig2

## TRYING TO USE GEOJSON GMAPS

In [93]:
# countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

# fig3 = gmaps.figure()

# pop_layer = gmaps.geojson_layer(countries_geojson)
# fig3.add_layer(pop_layer)
# fig3

In [15]:
# test_df = map_df[["Country", "Population"]]
# test2_df = test_df.set_index("Country")
# test2_df

In [16]:
# rows = test2_df["Population"]
# country_pop = dict(rows)
# #country_pop
# print(country_pop["Cuba"])

In [29]:
map_max=map_df["Population"].max()
map_min=map_df["Population"].min()
map_min=int(map_min) #Min is 809
map_max=int(map_max) #Max is 1439323774
map_mean=map_df["Population"].mean() #Mean is 39380459.4

In [51]:
import plotly.express as px
import plotly.graph_objects as go

In [87]:
fig = px.choropleth(map_df,  # Input Pandas DataFrame
                    locations="Country",  # DataFrame column with locations
                    color="Deaths",  # DataFrame column with color values
                    locationmode = 'country names', # Set to plot as world
                    scope = 'world',
                    color_continuous_scale= "blues",
                    
                    range_color=(map_df["Deaths"].min(), map_df["Deaths"].max()),
                    color_continuous_midpoint=(map_df["Deaths"].median())
                   )
# fig.update_layout(
#     title_text = 'State Rankings', # Create a Title
#     geo_scope='usa',  # Plot only the USA instead of globe
# )
fig.update_geos(projection_type="equirectangular")
#Color_continuous_scale:
    # Bad: earth, viridis, thermal, pinkyl, geyser, px.colors.sequential.Plasma
    # Ok: balance, agsunset, picnic, portland, puor,piyg, rdbu
    # Good: blues, rainbow, RdYlGn, spectral
fig.show()  # Output the plot to the screen